# Le but de ce notebook est de:
### 1. Comparer les différentes méthodes pour des données réelles
### 2. Vérifier l'hypothèse du rang faible

# Importer les librairies

In [ ]:
import matplotlib.pyplot as plt
from utilities import extract_complete_matrix
import seaborn as sns
import pandas as pd
from convex_optimization import test_convex
import numpy as np
from gradient_descent_script import *
from alternating_minimization import predict_AltMin
from clustering_method import complete_matrix

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Importer nos données

In [ ]:
df=pd.read_csv('data.csv', names=['movie_id', 'user_id','rating','date'])
# enlever la colonne date
df.drop('date', axis=1, inplace=True)

## Extraire nos matrices complètes

In [ ]:
M1, M2=extract_complete_matrix(df,0.3), extract_complete_matrix(df,0.25)

In [ ]:
shape_list=[M1.shape, M2.shape]

In [ ]:
shape_list

# Prédiction sur les matrices complètes

In [ ]:
predicted=[]
for M in [M1,M2]:
    m=np.sqrt(M.size)
    number_of_samples=int(m*np.log(m)) #Nombre d'entrées observées
    n,p=M.shape
    rank=int(min(n,p)**(1/5))
    #On choisit un ensemble d'entrées observées aléatoire identique pour toutes les méthodes
    omega=generate_omega(n,p,number_of_samples)
    #Descente de gradient
    predicted.append(generate_prediction_noprint(M,omega,rank=rank))
    #Minimisation alternée
    M_alt=generate_M(M,omega)
    predicted.append(predict_AltMin(M_alt,n,p,k=rank)) 
    #Optimisation convexe
    predicted.append(test_convex(M,omega))
    #Clustering
    M_clust=np.zeros_like(M)
    for i,j in omega:
        M_clust[i,j]=M[i,j]
    M_clust=(complete_matrix(M_clust))
    predicted.append(M_clust)

In [ ]:
#Les prédictions pour la première matrice
predicted1=predicted[:4]
#Les prédictions pour la deuxième matrice
predicted2=predicted[4:]

In [ ]:
#Les erreurs dans la prédiction pour la première matrice
RMSE1=[np.linalg.norm(M1-M)/np.sqrt(M.size) for M in predicted1]
#Les erreurs dans la prédiction pour la deuxième matrice
RMSE2=[np.linalg.norm(M2-M)/np.sqrt(M.size) for M in predicted2]

In [ ]:
labels = ['Descente\n du gradient', 'Minimisation\n alternée', 'Optimisation\n convexe', 'Clustering']
plt.bar(labels,RMSE1)
plt.xlabel('Méthodes')
plt.ylabel('RMSE')
plt.title('RMSE pour des données réelles (Matrice de taille 1309x41)')
plt.show()

In [ ]:
plt.bar(labels,RMSE2)
plt.xlabel('Méthodes')
plt.ylabel('RMSE')
plt.title('RMSE pour des données réelles (Matrice de taille 298x87)')
plt.show()

# Distribution des valeurs singulières

## Préparer nos données

In [ ]:
data_list=[extract_complete_matrix(df,alpha) for alpha in [0.3,0.25,0.2,0.15]]

In [ ]:
shape_list=[str(M.shape) for M in data_list]

## Calculer les valeurs singulières

In [ ]:
svd_list=[sorted(np.linalg.svd(M,compute_uv=False), reverse=True) for M in data_list]
svd_list=[100*np.array(svd)/np.sum(svd) for svd in svd_list]

## Visualiser la distribution des valeurs singulières

In [ ]:
plt.grid(True)
for svd in svd_list:
    sns.scatterplot(svd,s=20)
plt.xlabel("Valeurs singulières")
plt.ylabel("Pourcentage de la norme nucléaire")
plt.yscale('log')
plt.title('Distribution des valeurs singulières')
plt.legend(shape_list)
plt.show()